## Topic Modelling using Gensim

In [27]:
import nltk; nltk.download('stopwords')
import os

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy

import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

# LDA Model

In [12]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [4]:
data = pd.read_csv('joinedLineItems.csv', encoding='utf-8')

#print(data.head())

data = data['Description'].dropna()

print(data.head())

0    WHITE HANGING HEART T-LIGHT HOLDER WHITE METAL...
1     HAND WARMER UNION JACK HAND WARMER RED POLKA DOT
2    ASSORTED COLOUR BIRD ORNAMENT POPPY'S PLAYHOUS...
3    JAM MAKING SET WITH JARS RED COAT RACK PARIS F...
4                             BATH BUILDING BLOCK WORD
Name: Description, dtype: object


In [6]:
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
data = [re.sub('\s+', ' ', sent) for sent in data]
data = [re.sub("\'", "", sent) for sent in data]
pprint(data[:1])

['WHITE HANGING HEART T-LIGHT HOLDER WHITE METAL LANTERN CREAM CUPID HEARTS '
 'COAT HANGER KNITTED UNION FLAG HOT WATER BOTTLE RED WOOLLY HOTTIE WHITE '
 'HEART. SET 7 BABUSHKA NESTING BOXES GLASS STAR FROSTED T-LIGHT HOLDER']


###### Tokenize and Clean Text

In [8]:
def sent_to_words(sentences):
    for sentence in sentences:
                yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['white', 'hanging', 'heart', 'light', 'holder', 'white', 'metal', 'lantern', 'cream', 'cupid', 'hearts', 'coat', 'hanger', 'knitted', 'union', 'flag', 'hot', 'water', 'bottle', 'red', 'woolly', 'hottie', 'white', 'heart', 'set', 'babushka', 'nesting', 'boxes', 'glass', 'star', 'frosted', 'light', 'holder']]


In [9]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['white', 'hanging', 'heart', 'light', 'holder', 'white', 'metal', 'lantern', 'cream', 'cupid', 'hearts', 'coat', 'hanger', 'knitted', 'union', 'flag', 'hot', 'water', 'bottle', 'red', 'woolly_hottie', 'white', 'heart', 'set', 'babushka', 'nesting', 'boxes', 'glass', 'star', 'frosted', 'light', 'holder']


In [10]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [15]:
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['hang', 'heart', 'light', 'holder', 'white', 'metal', 'knit', 'flag', 'hot', 'water', 'bottle', 'red', 'woolly_hottie', 'white', 'heart', 'set', 'nest', 'glass', 'star', 'frost', 'light', 'holder']]


In [16]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (6, 2), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 2), (17, 1)]]


In [17]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('bottle', 1),
  ('flag', 1),
  ('frost', 1),
  ('glass', 1),
  ('hang', 1),
  ('heart', 2),
  ('holder', 2),
  ('hot', 1),
  ('knit', 1),
  ('light', 2),
  ('metal', 1),
  ('nest', 1),
  ('red', 1),
  ('set', 1),
  ('star', 1),
  ('water', 1),
  ('white', 2),
  ('woolly_hottie', 1)]]

In [46]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [43]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]



[(0,
  '0.085*"set" + 0.032*"vintage" + 0.025*"card" + 0.020*"cake" + '
  '0.018*"design" + 0.018*"pack" + 0.017*"box" + 0.017*"pink" + 0.015*"paper" '
  '+ 0.015*"kit"'),
 (1,
  '0.137*"bag" + 0.073*"hot" + 0.072*"water" + 0.068*"bottle" + 0.036*"jumbo" '
  '+ 0.033*"vintage" + 0.026*"warm" + 0.025*"lunch" + 0.023*"red" + '
  '0.022*"pink"'),
 (2,
  '0.080*"heart" + 0.044*"light" + 0.035*"metal" + 0.031*"hang" + '
  '0.029*"decoration" + 0.028*"holder" + 0.027*"sign" + 0.022*"home" + '
  '0.020*"wooden" + 0.019*"small"')]


In [44]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -5.651890385139709

Coherence Score:  0.4423261799043243


In [45]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.032989 -0.267672       1        1  52.313744
2     -0.254575  0.158268       2        1  33.441128
1      0.287564  0.109404       3        1  14.245126, topic_info=          Term          Freq         Total Category  logprob  loglift
81         bag  26312.000000  26312.000000  Default  30.0000  30.0000
5        heart  39974.000000  39974.000000  Default  29.0000  29.0000
13         set  61683.000000  61683.000000  Default  28.0000  28.0000
7          hot  15016.000000  15016.000000  Default  27.0000  27.0000
15       water  14334.000000  14334.000000  Default  26.0000  26.0000
..         ...           ...           ...      ...      ...      ...
141  chocolate   1636.049316   4367.677734   Topic3  -4.7689   0.9668
25        love   1753.121460   8945.975586   Topic3  -4.6998   0.3189
54   retrospot   1628.697266   8806.322266   Topic3  -4.7734   0.2611
42       dolly   1548.756958   8205.975586   Topic3  -4.8237   0.2813
60         tea   1503.793213  10594.709961   Topic3  -4.8532  -0.0036

[202 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
1604      1  0.034190  accessory
1604      2  0.057781  accessory
1604      3  0.908077  accessory
138       1  0.000751    antique
138       2  0.999243    antique
...     ...       ...        ...
484       1  0.999215       wrap
484       2  0.000200       wrap
484       3  0.000798       wrap
137       1  0.000167       zinc
137       2  0.999902       zinc

[458 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2])

### LDA Mallet Model

In [29]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
os.environ.update({'MALLET_HOME':r'C:/new_mallet/mallet-2.0.8/'})
mallet_path = r'C:/new_mallet/mallet-2.0.8/bin/mallet' # update this path

ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

CalledProcessError: Command 'C:/new_mallet/mallet-2.0.8/bin/mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input C:\Users\david\AppData\Local\Temp\229cfc_corpus.txt --output C:\Users\david\AppData\Local\Temp\229cfc_corpus.mallet' returned non-zero exit status 1.